# Experiment 1: Zero-Shot Prompt Engineering

**Objective:** Evaluate LLaMA-3.1-8B-Instruct on Solana smart contract vulnerability detection without any training.

**Method:** Role-based prompting where the model acts as a security analyzer.

**Reference:** Boi, B., & Esposito, C. (2025). Prompt Engineering vs. Fine-Tuning for LLM-Based Vulnerability Detection.

---

## 1. Environment Setup

In [ ]:
import torch
import os
import warnings

# Suppress all warnings for clean output
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("=" * 50)
print("ENVIRONMENT CHECK")
print("=" * 50)

if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    GPU_MEMORY = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {GPU_NAME}")
    print(f"Memory: {GPU_MEMORY:.1f} GB")
    print("Status: Ready")
else:
    print("ERROR: GPU not detected!")
    print("Go to: Settings -> Accelerator -> GPU T4 x2")

In [ ]:
%%capture
# Install packages 
!pip install -q bitsandbytes accelerate

In [ ]:
print("Packages installed successfully.")

## 2. Imports & Authentication

In [ ]:
import json
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
from huggingface_hub import login

# Suppress logging
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("accelerate").setLevel(logging.ERROR)

# Authentication
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
HF_TOKEN = secrets.get_secret("HF_TOKEN")
login(token=HF_TOKEN, add_to_git_credential=False)

print("Authentication successful.")

## 3. Load Dataset

In [ ]:
# Find dataset path
POSSIBLE_PATHS = [
    "/kaggle/input/solana-dataset/solana_140s_final.json",
    "/kaggle/input/solana_140s_final.json",
    "/kaggle/working/solana_140s_final.json"
]

DATASET_PATH = None
for path in POSSIBLE_PATHS:
    if os.path.exists(path):
        DATASET_PATH = path
        break

if DATASET_PATH is None:
    raise FileNotFoundError("Dataset not found. Please upload solana_140s_final.json")

with open(DATASET_PATH, 'r') as f:
    dataset = json.load(f)

print(f"Dataset: {len(dataset)} samples")
print(f"Path: {DATASET_PATH}")
print("\nVulnerability Types:")
for vtype, count in sorted(Counter(s['vulnerability_type'] for s in dataset).items()):
    print(f"  {vtype}: {count}")
print(f"\nLabels: VULNERABLE={sum(1 for s in dataset if s['label']=='VULNERABLE')}, SAFE={sum(1 for s in dataset if s['label']=='SAFE')}")

## 4. Data Split

In [ ]:
# Stratified split by vulnerability type (80% train, 10% val, 10% test)
by_vuln_type = defaultdict(list)
for sample in dataset:
    by_vuln_type[sample['vulnerability_type']].append(sample)

train_data, val_data, test_data = [], [], []

for vtype, samples in by_vuln_type.items():
    labels = [s['label'] for s in samples]
    train_samples, temp_samples = train_test_split(samples, test_size=0.2, stratify=labels, random_state=42)
    temp_labels = [s['label'] for s in temp_samples]
    val_samples, test_samples = train_test_split(temp_samples, test_size=0.5, stratify=temp_labels, random_state=42)
    train_data.extend(train_samples)
    val_data.extend(val_samples)
    test_data.extend(test_samples)

print("Data Split:")
print(f"  Train: {len(train_data)} (80%)")
print(f"  Val:   {len(val_data)} (10%)")
print(f"  Test:  {len(test_data)} (10%)")

## 5. Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

# 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quant_config,
    device_map="auto",
    token=HF_TOKEN,
    low_cpu_mem_usage=True
)

print(f"\nModel: {MODEL_ID}")
print(f"Parameters: {model.num_parameters():,}")
print(f"Quantization: 4-bit NF4")

## 6. Zero-Shot Prediction Function

**Method:** The model receives a role-based prompt describing its task as a security analyzer, then classifies the code without any prior training examples.

In [ ]:
# System prompt for security analysis role
SYSTEM_PROMPT = """You are a smart contract security analyzer.
You analyze Solana smart contracts written in Rust and identify vulnerabilities.
Classify the code as either VULNERABLE or SAFE.
Respond with only one word: VULNERABLE or SAFE."""

def extract_code(sample):
    """Extract code content from formatted sample."""
    text = sample['text']
    start_marker = '<|start_header_id|>user<|end_header_id|>'
    end_marker = '<|eot_id|><|start_header_id|>assistant'
    
    start_idx = text.find(start_marker)
    end_idx = text.find(end_marker)
    
    if start_idx != -1 and end_idx != -1:
        return text[start_idx + len(start_marker):end_idx].strip()
    return text[:1000]

def predict_zero_shot(sample):
    """Zero-shot prediction using role-based prompting."""
    code = extract_code(sample)
    
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>

Analyze this Solana smart contract:

{code}

Is this code VULNERABLE or SAFE?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip().upper()
    first_word = response.split()[0] if response.split() else ""
    
    return 'VULNERABLE' if 'VULN' in first_word else 'SAFE'

print("Zero-Shot prediction function ready.")
print(f"\nMethod: Role-based prompting")
print(f"Training: None (zero-shot)")

## 7. Evaluation

In [ ]:
print("=" * 50)
print("RUNNING EVALUATION")
print("=" * 50)

results = []
for sample in tqdm(test_data, desc="Evaluating"):
    pred = predict_zero_shot(sample)
    results.append({
        'vulnerability_type': sample['vulnerability_type'],
        'ground_truth': sample['label'],
        'prediction': pred,
        'correct': sample['label'] == pred
    })

print("\nEvaluation complete.")

## 8. Results

In [ ]:
# Calculate metrics per vulnerability type
metrics_by_type = {}
vuln_types = sorted(set(r['vulnerability_type'] for r in results))

for vtype in vuln_types:
    type_results = [r for r in results if r['vulnerability_type'] == vtype]
    gt = [r['ground_truth'] for r in type_results]
    pred = [r['prediction'] for r in type_results]
    
    metrics_by_type[vtype] = {
        'Accuracy': round(accuracy_score(gt, pred), 2),
        'Precision': round(precision_score(gt, pred, pos_label='VULNERABLE', zero_division=0), 2),
        'Recall': round(recall_score(gt, pred, pos_label='VULNERABLE', zero_division=0), 2),
        'F1-score': round(f1_score(gt, pred, pos_label='VULNERABLE', zero_division=0), 2)
    }

# Calculate averages
avg_metrics = {
    'Accuracy': round(sum(m['Accuracy'] for m in metrics_by_type.values()) / len(metrics_by_type), 2),
    'Precision': round(sum(m['Precision'] for m in metrics_by_type.values()) / len(metrics_by_type), 2),
    'Recall': round(sum(m['Recall'] for m in metrics_by_type.values()) / len(metrics_by_type), 2),
    'F1-score': round(sum(m['F1-score'] for m in metrics_by_type.values()) / len(metrics_by_type), 2)
}

# Display
print("=" * 70)
print("RESULTS: Experiment 1 - Zero-Shot Prompt Engineering")
print("=" * 70)
print(f"{'Vulnerability':<20} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-score':<12}")
print("-" * 70)
for vtype in vuln_types:
    m = metrics_by_type[vtype]
    print(f"{vtype:<20} {m['Accuracy']:<12} {m['Precision']:<12} {m['Recall']:<12} {m['F1-score']:<12}")
print("-" * 70)
print(f"{'Average':<20} {avg_metrics['Accuracy']:<12} {avg_metrics['Precision']:<12} {avg_metrics['Recall']:<12} {avg_metrics['F1-score']:<12}")
print("=" * 70)

## 9. Confusion Matrix

In [ ]:
all_gt = [r['ground_truth'] for r in results]
all_pred = [r['prediction'] for r in results]
cm = confusion_matrix(all_gt, all_pred, labels=['VULNERABLE', 'SAFE'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['VULNERABLE', 'SAFE'],
            yticklabels=['VULNERABLE', 'SAFE'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Experiment 1: Zero-Shot PE - Confusion Matrix')
plt.tight_layout()
plt.savefig('/kaggle/working/cm_zero_shot.png', dpi=150)
plt.show()

print(f"\nConfusion Matrix:")
print(f"  TP (detected vulnerabilities): {cm[0,0]}")
print(f"  FN (missed vulnerabilities):   {cm[0,1]}")
print(f"  FP (false alarms):             {cm[1,0]}")
print(f"  TN (correct safe):             {cm[1,1]}")

## 10. Save Results

In [ ]:
# Save CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/kaggle/working/results_zero_shot.csv', index=False)

# Save summary JSON
summary = {
    'experiment': 'Zero-Shot Prompt Engineering',
    'experiment_id': 1,
    'model': {
        'name': 'Llama-3.1-8B-Instruct',
        'quantization': '4-bit NF4',
        'parameters': '8B'
    },
    'method': {
        'type': 'Zero-Shot',
        'description': 'Role-based prompting without training',
        'training_required': False
    },
    'dataset': {
        'total': len(dataset),
        'train': len(train_data),
        'val': len(val_data),
        'test': len(test_data),
        'vulnerability_types': 7
    },
    'results': {
        'overall_accuracy': round(accuracy_score(all_gt, all_pred), 4),
        'per_vulnerability': metrics_by_type,
        'average': avg_metrics
    },
    'confusion_matrix': {
        'TP': int(cm[0,0]),
        'FN': int(cm[0,1]),
        'FP': int(cm[1,0]),
        'TN': int(cm[1,1])
    }
}

with open('/kaggle/working/summary_zero_shot.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("Files saved:")
print("  - results_zero_shot.csv")
print("  - summary_zero_shot.json")
print("  - cm_zero_shot.png")

In [ ]:
print("\n" + "=" * 50)
print("EXPERIMENT 1 COMPLETE")
print("=" * 50)
print(f"\nOverall Accuracy: {summary['results']['overall_accuracy']:.2%}")
print(f"Average F1-Score: {avg_metrics['F1-score']}")
print(f"\nMethod: Zero-Shot (no training)")
print(f"Model: LLaMA-3.1-8B-Instruct")